# Imports

In [79]:
import glob
import os

import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sns

## Def describe

In [25]:
def describe(df, random=False):
    print(df.shape)
    print("--- First 5 entries ---")
    display(df.head())
    if random:
        print('--- Random subset ---')
        display(df.sample(5))

## Read one2one h5ad

In [33]:
adata = sc.read(
#     "/home/phoenix/data_sm/code/immune-evolution/h5ads/human-lemur-mouse-bat__lung_only_copy.h5ad"
    "/home/olga/data_sm/immune-evolution/h5ads/human-lemur-mouse-bat/human-lemur-mouse-bat__lung_only.h5ad"
)
adata.obs = adata.obs.reset_index().set_index('cell_id')
SHARED_CELLTYPES = ['Alveolar Epithelial Type 2',
 'B cell',
 'Capillary',
 'Dendritic',
 'Fibroblast',
 'Macrophage',
 'Monocyte',
 'Natural Killer T cell',
 'Smooth Muscle and Myofibroblast',
 'T cell']
len(SHARED_CELLTYPES)

adata_shared = adata[adata.obs.broad_group.isin(SHARED_CELLTYPES)]
adata_shared

/home/olga/miniconda3/envs/immune-evolution/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


View of AnnData object with n_obs × n_vars = 89795 × 10560
    obs: 'index', 'age', 'cell_barcode', 'cell_ontology_class', 'cell_ontology_id', 'channel', 'free_annotation', 'individual', 'sample', 'sequencing_run', 'sex', 'species', 'species_batch', 'species_latin', 'tissue', 'narrow_group', 'broad_group', 'compartment_group', 'compartment_narrow', 'channel_cleaned', 'batch', 'n_genes', 'n_counts', 'species_batch_v2', 'compartment_broad', 'compartment_broad_narrow', 'compartment_species', 'compartment_narrow_species', 'common_individual_id'
    var: 'bat__gene_name', 'mouse_lemur__gene_name-bat', 'mouse__gene_name-bat', 'mouse_lemur__gene_name_x-hlm', 'mouse__gene_name_x-hlm', 'gene_ids-lemur-hlm', 'n_cells-mouse-hlm', 'mouse_lemur__gene_name_y-hlm', 'mouse__gene_name_y-hlm'

# Read parquet files

## File paths

In [227]:
sketch_id = 'alphabet-dayhoff__ksize-51__scaled-10'
sig_outdir_base = "/home/olga/data_sm/immune-evolution/kmer-signatures"


parquets = glob.glob(f'/home/olga/data_sm/immune-evolution/kmer-signatures/*/2--single-cell-kmers/{sketch_id}/hash2kmer__unique_kmers_per_celltype.parquet')
parquets

['/home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/3--test-bat/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/4--test-lemur/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet']

In [228]:
# parquets = glob.glob(f'/home/olga/data_sm/immune-evolution/kmer-signatures/*/2--single-cell-kmers/*/hash2kmer__unique_kmers_per_celltype.parquet')
# parquets

In [229]:
%%time
dfs = []

for parquet in parquets:
    ! ls -lha $parquet
    %time df = pd.read_parquet(parquet)
    df = df.rename(columns={'broad_group': 'groundtruth_celltype'})
    dfs.append(df)
    
hash2kmer = pd.concat(dfs)
describe(hash2kmer)

-rw-r--r-- 1 olga czb 2.4G May  3 11:03 /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet
CPU times: user 1min 2s, sys: 40.4 s, total: 1min 43s
Wall time: 2min 56s
-rw-r--r-- 1 olga czb 5.0G May 10 13:26 /home/olga/data_sm/immune-evolution/kmer-signatures/3--test-bat/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet
CPU times: user 2min 53s, sys: 1min 3s, total: 3min 56s
Wall time: 3min 52s
-rw-r--r-- 1 olga czb 149M Apr 30 13:26 /home/olga/data_sm/immune-evolution/kmer-signatures/4--test-lemur/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet
CPU times: user 3.89 s, sys: 2.02 s, total: 5.9 s
Wall time: 4.11 s
-rw-r--r-- 1 olga czb 3.3G May  3 11:12 /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/2--single-cell-kmers/alphabet-dayhoff__ksize-51

,kmer_in_sequence,kmer_in_alphabet,hashval,read_name,gene_name,species,cell_id,sketch_id,moltype,ksize,scaled,alignment_status,groundtruth_celltype
0,DLNVKPKTIKTLEENLG,cecedbdbedbeccceb,550348301278027135,A00111:155:HFCWYDMXX:1:1359:1190:25066\tCB:Z:G...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast
1,IKTLEENLGITIQDIGV,edbecccebebeccebe,974148631326387115,A00111:155:HFCWYDMXX:1:1359:1190:25066\tCB:Z:G...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast
3,CRKLKLDPFLTPYTKIN,addedecbfebbfbdec,1662986983738689810,A00111:155:HFCWYDMXX:2:1129:25961:14544\tCB:Z:...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast
4,RKLKLDPFLTPYTKINS,ddedecbfebbfbdecb,799720167227311772,A00111:155:HFCWYDMXX:2:1129:25961:14544\tCB:Z:...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast
7,LDPFLTPYTKINSRWIK,ecbfebbfbdecbdfed,537198855570371996,A00111:155:HFCWYDMXX:2:1360:14886:30185\tCB:Z:...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast


CPU times: user 7min 13s, sys: 4min 15s, total: 11min 28s
Wall time: 13min 42s


## Read predicted cell types

In [230]:
predicted_cells_min_5_percent = pd.read_parquet(
    os.path.join(sig_outdir_base, "aggregated-predicted-cells.parquet")
)
predicted_cells_min_5_percent = predicted_cells_min_5_percent.query('sketch_id == @sketch_id')
describe(predicted_cells_min_5_percent)

(29750, 43)
--- First 5 entries ---


,similarity,mouse1_broad_group,search_method,database_type,sketch_id,mol_or_alpha,alphabet,ksize,sketch_style,sketch_value,...,bat_narrow_group,bat_broad_group,bat_compartment_group,human_channel,human_cell_ontology_class,human_narrow_group,human_broad_group,human_compartment_group,predicted_compartment,groundtruth_compartment
query_cell_id,,,,,,,,,,,,,,,,,,,,,
MACA_18m_F_LUNG_51__AAACCTGCAGCCTATA,0.119992,T cell,similarity_abundance,min-5-percent-shared-kmers-remove-common-kmers,alphabet-dayhoff__ksize-51__scaled-10,alphabet,dayhoff,51,scaled,10,...,None,None,None,None,None,None,None,None,lymphoid,lymphoid
MACA_18m_F_LUNG_51__AAACCTGGTGTGGTTT,0.170558,B cell,similarity_abundance,min-5-percent-shared-kmers-remove-common-kmers,alphabet-dayhoff__ksize-51__scaled-10,alphabet,dayhoff,51,scaled,10,...,None,None,None,None,None,None,None,None,lymphoid,lymphoid
MACA_18m_F_LUNG_51__AAACGGGAGTTTCCTT,0.080257,T cell,similarity_abundance,min-5-percent-shared-kmers-remove-common-kmers,alphabet-dayhoff__ksize-51__scaled-10,alphabet,dayhoff,51,scaled,10,...,None,None,None,None,None,None,None,None,lymphoid,lymphoid
MACA_18m_F_LUNG_51__AAACGGGCAGTTCATG,0.263881,Monocyte,similarity_abundance,min-5-percent-shared-kmers-remove-common-kmers,alphabet-dayhoff__ksize-51__scaled-10,alphabet,dayhoff,51,scaled,10,...,None,None,None,None,None,None,None,None,myeloid,myeloid
MACA_18m_F_LUNG_51__AAACGGGGTCATCCCT,0.326669,Monocyte,similarity_abundance,min-5-percent-shared-kmers-remove-common-kmers,alphabet-dayhoff__ksize-51__scaled-10,alphabet,dayhoff,51,scaled,10,...,None,None,None,None,None,None,None,None,myeloid,myeloid


### Join hash2kmer with predicted cell types

In [231]:
celltype_predictions = predicted_cells_min_5_percent[
    ["predicted_celltype", "groundtruth_compartment", "predicted_compartment"]
]
celltype_predictions.index = celltype_predictions.index.str.upper()
describe(celltype_predictions)

(29750, 3)
--- First 5 entries ---


,predicted_celltype,groundtruth_compartment,predicted_compartment
query_cell_id,,,
MACA_18M_F_LUNG_51__AAACCTGCAGCCTATA,T cell,lymphoid,lymphoid
MACA_18M_F_LUNG_51__AAACCTGGTGTGGTTT,B cell,lymphoid,lymphoid
MACA_18M_F_LUNG_51__AAACGGGAGTTTCCTT,T cell,lymphoid,lymphoid
MACA_18M_F_LUNG_51__AAACGGGCAGTTCATG,Monocyte,myeloid,myeloid
MACA_18M_F_LUNG_51__AAACGGGGTCATCCCT,Monocyte,myeloid,myeloid


In [232]:
hash2kmer.cell_id = hash2kmer.cell_id.str.upper()

In [233]:
hash2kmer_with_predictions = hash2kmer.join(celltype_predictions, on='cell_id')
describe(hash2kmer_with_predictions)

(148123511, 16)
--- First 5 entries ---


,kmer_in_sequence,kmer_in_alphabet,hashval,read_name,gene_name,species,cell_id,sketch_id,moltype,ksize,scaled,alignment_status,groundtruth_celltype,predicted_celltype,groundtruth_compartment,predicted_compartment
0,DLNVKPKTIKTLEENLG,cecedbdbedbeccceb,550348301278027135,A00111:155:HFCWYDMXX:1:1359:1190:25066\tCB:Z:G...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast,Fibroblast,stromal,stromal
1,IKTLEENLGITIQDIGV,edbecccebebeccebe,974148631326387115,A00111:155:HFCWYDMXX:1:1359:1190:25066\tCB:Z:G...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast,Fibroblast,stromal,stromal
3,CRKLKLDPFLTPYTKIN,addedecbfebbfbdec,1662986983738689810,A00111:155:HFCWYDMXX:2:1129:25961:14544\tCB:Z:...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast,Fibroblast,stromal,stromal
4,RKLKLDPFLTPYTKINS,ddedecbfebbfbdecb,799720167227311772,A00111:155:HFCWYDMXX:2:1129:25961:14544\tCB:Z:...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast,Fibroblast,stromal,stromal
7,LDPFLTPYTKINSRWIK,ecbfebbfbdecbdfed,537198855570371996,A00111:155:HFCWYDMXX:2:1360:14886:30185\tCB:Z:...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast,Fibroblast,stromal,stromal


In [234]:
hash2kmer_with_predictions['hashval'] = hash2kmer_with_predictions['hashval'].astype(str)

### Write joined to parquet

In [235]:
hash2kmer_with_predictions.to_parquet(
    os.path.join(sig_outdir_base, f"aggregated-hash2kmer-with-predicted-cells__{sketch_id}.parquet")
)

## Seems like some cell ids aren't there .. why?

In [236]:
hash2kmer_cell_ids = set(hash2kmer.cell_id)

In [245]:
adata_shared.obs.index = adata_shared.obs.index.str.upper()

In [246]:
cell_ids_not_in_hash2kmer = adata_shared.obs.index.difference(hash2kmer_cell_ids)
len(cell_ids_not_in_hash2kmer)

61250

In [247]:
adata_shared.obs.loc[cell_ids_not_in_hash2kmer].species.value_counts()

Human          27521
Mouse lemur    18284
Mouse          15372
Bat               73
Name: species, dtype: int64

In [248]:
channel_sizes = adata_shared.obs.groupby('channel').size()

In [249]:
no_id_present_channel_sizes = adata_shared.obs.loc[cell_ids_not_in_hash2kmer].groupby('channel').size()

In [250]:
no_id_present_channel_sizes

channel
10X_P1_6                  7680
10X_P1_14                 1413
10X_P2_5                   825
10X_P3_0                   447
10X_P5_13                  936
10X_P5_14                 1202
10X_P7_8                   294
10X_P7_9                   745
10X_P8_12                  489
10X_P8_13                 1327
Antoine_Lung_CD31_10X     5873
Antoine_Lung_EPCAM_10X    2647
Antoine_Lung_P3_10X       3547
Bernard_Lung_10X          2596
MACA_18m_F_LUNG_50          14
MACA_18m_F_LUNG_51           0
MACA_18m_M_LUNG_52           0
MACA_18m_M_LUNG_53           0
MACA_21m_F_LUNG_54           0
MACA_21m_F_LUNG_55           0
Martine_Lung_1_10X         856
Martine_Lung_2_10X         887
P1_2                      1677
P1_3                      1932
P1_4                      2265
P2_1                      2921
P2_2                      2764
P2_3                      2328
P2_4                       882
P2_5                      3817
P2_6                       922
P2_7                      2953


In [251]:
100 * no_id_present_channel_sizes/channel_sizes

channel
10X_P1_6                  100.000000
10X_P1_14                 100.000000
10X_P2_5                  100.000000
10X_P3_0                  100.000000
10X_P5_13                 100.000000
10X_P5_14                 100.000000
10X_P7_8                  100.000000
10X_P7_9                  100.000000
10X_P8_12                 100.000000
10X_P8_13                 100.000000
Antoine_Lung_CD31_10X     100.000000
Antoine_Lung_EPCAM_10X    100.000000
Antoine_Lung_P3_10X        81.521489
Bernard_Lung_10X          100.000000
MACA_18m_F_LUNG_50          5.000000
MACA_18m_F_LUNG_51          0.000000
MACA_18m_M_LUNG_52          0.000000
MACA_18m_M_LUNG_53          0.000000
MACA_21m_F_LUNG_54          0.000000
MACA_21m_F_LUNG_55          0.000000
Martine_Lung_1_10X        100.000000
Martine_Lung_2_10X        100.000000
P1_2                      100.000000
P1_3                      100.000000
P1_4                      100.000000
P2_1                      100.000000
P2_2                      100.

In [254]:
celltype_hashvals = hash2kmer.groupby(['species', 'groundtruth_celltype']).alignment_status.value_counts()
celltype_hashvals.head()

species  groundtruth_celltype        alignment_status
bat      Alveolar Epithelial Type 2  aligned             13522422
                                     unaligned             759922
         B cell                      aligned              1055558
                                     unaligned              58700
         Capillary                   aligned             14688263
Name: alignment_status, dtype: int64

In [255]:
celltype_hashvals

species  groundtruth_celltype             alignment_status
bat      Alveolar Epithelial Type 2       aligned             13522422
                                          unaligned             759922
         B cell                           aligned              1055558
                                          unaligned              58700
         Capillary                        aligned             14688263
                                                                ...   
mouse    Natural Killer T cell            unaligned              42411
         Smooth Muscle and Myofibroblast  aligned               184572
                                          unaligned               5618
         T cell                           aligned              5653466
                                          unaligned             189747
Name: alignment_status, Length: 78, dtype: int64

In [256]:
celltype_hashval_percentage = celltype_hashvals.groupby(level=[0, 1]).apply(lambda x: 100 * x/x.sum())
celltype_hashval_percentage.name = 'percentage'
celltype_hashval_percentage = celltype_hashval_percentage.reset_index()
celltype_hashval_percentage

,species,groundtruth_celltype,alignment_status,percentage
0,bat,Alveolar Epithelial Type 2,aligned,94.679291
1,bat,Alveolar Epithelial Type 2,unaligned,5.320709
2,bat,B cell,aligned,94.731920
3,bat,B cell,unaligned,5.268080
4,bat,Capillary,aligned,95.639091
...,...,...,...,...
73,mouse,Natural Killer T cell,unaligned,3.501037
74,mouse,Smooth Muscle and Myofibroblast,aligned,97.046112
75,mouse,Smooth Muscle and Myofibroblast,unaligned,2.953888
76,mouse,T cell,aligned,96.752694


In [257]:
celltype_hashval_percentage_unaligned = celltype_hashval_percentage.query('alignment_status == "unaligned"')


In [259]:
order = celltype_hashval_percentage_unaligned.query('species == "bat"').sort_values(
    "percentage"
)["groundtruth_celltype"]
order

9                          Fibroblast
5                           Capillary
17    Smooth Muscle and Myofibroblast
11                         Macrophage
19                             T cell
13                           Monocyte
15              Natural Killer T cell
3                              B cell
1          Alveolar Epithelial Type 2
7                           Dendritic
Name: groundtruth_celltype, dtype: category
Categories (10, object): ['Alveolar Epithelial Type 2', 'B cell', 'Capillary', 'Dendritic', ..., 'Monocyte', 'Natural Killer T cell', 'Smooth Muscle and Myofibroblast', 'T cell']

In [ ]:
g = sns.catplot(
    data=celltype_hashval_percentage_unaligned,
    y="groundtruth_celltype",
    x="percentage",
    hue="species",
    kind="bar",
    order=order
)
g.set(xscale="log")

In [ ]:
g = sns.catplot(
    data=celltype_hashval_percentage_unaligned,
    y="groundtruth_celltype",
    x="percentage",
    col="species",
    kind="bar",
    order=order
)
g.set(xscale="log")

## Read mouse diagnostic k-mer csvs

In [261]:
mouse_dir = '/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/'
celltype_kmer_subdir = '5--celltype-kmers--merged-celltype-remove-common-kmers--min-kmer-count--5-percent'

mouse_celltype_kmer_csvs = glob.glob(os.path.join(mouse_dir, celltype_kmer_subdir, sketch_id, 'csvs', '*csv'))
mouse_celltype_kmer_csvs

['/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/5--celltype-kmers--merged-celltype-remove-common-kmers--min-kmer-count--5-percent/alphabet-dayhoff__ksize-51__scaled-10/csvs/natural_killer_t_cell.csv',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/5--celltype-kmers--merged-celltype-remove-common-kmers--min-kmer-count--5-percent/alphabet-dayhoff__ksize-51__scaled-10/csvs/alveolar_epithelial_type_2.csv',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/5--celltype-kmers--merged-celltype-remove-common-kmers--min-kmer-count--5-percent/alphabet-dayhoff__ksize-51__scaled-10/csvs/smooth_muscle_and_myofibroblast.csv',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/5--celltype-kmers--merged-celltype-remove-common-kmers--min-kmer-count--5-percent/alphabet-dayhoff__ksize-51__scaled-10/csvs/monocyte.csv',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/5--celltype-kmers--merged-celltype-remove

In [262]:
%%time

dfs = []

for csv in mouse_celltype_kmer_csvs:
    basename = os.path.basename(csv)
    print('\ncelltype file:', )
    %time df = pd.read_csv(csv)
    df['celltype'] = basename.split('.')[0]
    dfs.append(df)
mouse_celltype_kmers = pd.concat(dfs)
describe(mouse_celltype_kmers)

natural_killer_t_cell.csv
CPU times: user 1.84 s, sys: 182 ms, total: 2.02 s
Wall time: 2.54 s
alveolar_epithelial_type_2.csv
CPU times: user 1.51 s, sys: 142 ms, total: 1.65 s
Wall time: 2.06 s
smooth_muscle_and_myofibroblast.csv
CPU times: user 514 ms, sys: 60.8 ms, total: 575 ms
Wall time: 881 ms
monocyte.csv
CPU times: user 27.5 s, sys: 2.58 s, total: 30.1 s
Wall time: 35.9 s
b_cell.csv
CPU times: user 8.55 s, sys: 915 ms, total: 9.47 s
Wall time: 16.3 s
fibroblast.csv
CPU times: user 3.84 s, sys: 579 ms, total: 4.42 s
Wall time: 5.48 s
t_cell.csv
CPU times: user 5.14 s, sys: 518 ms, total: 5.66 s
Wall time: 6.77 s
capillary.csv
CPU times: user 11.8 s, sys: 1.35 s, total: 13.1 s
Wall time: 16 s
dendritic.csv
CPU times: user 2.54 s, sys: 253 ms, total: 2.8 s
Wall time: 3.4 s
macrophage.csv
CPU times: user 2.42 s, sys: 226 ms, total: 2.65 s
Wall time: 3.91 s
(22370943, 4)
--- First 5 entries ---


,kmer_in_sequence,kmer_in_alphabet,hashval,read_name
0,KLSSPRGGMKKKTTHFG,debbbdbbedddbbdfb,308103755003737144,A00111:73:H3YHNDMXX:2:1218:21983:15264\tCB:Z:A...
1,LSSPRGGMKKKTTHFGE,ebbbdbbedddbbdfbc,177719887282804153,A00111:73:H3YHNDMXX:2:1218:21983:15264\tCB:Z:A...
2,YTVGKRFKEANNFLRPF,fbebddfdcbccfedbf,1822116523473074621,A00111:73:H3YHNDMXX:1:2311:5367:19179\tCB:Z:AA...
3,KFGIICMEDLIQDIYTI,dfbeeaecceeccefbe,115722640331910096,A00111:73:H3YHNDMXX:2:1175:17463:20556\tCB:Z:A...
4,KFGIICMEDLIQEIYTV,dfbeeaecceeccefbe,115722640331910096,A00111:73:H3YHNDMXX:2:2229:8260:32111\tCB:Z:AA...


CPU times: user 1min 21s, sys: 21.4 s, total: 1min 42s
Wall time: 2min 31s


# Overlap in hashes for cell types

In [263]:
hash2kmer.head()

,kmer_in_sequence,kmer_in_alphabet,hashval,read_name,gene_name,species,cell_id,sketch_id,moltype,ksize,scaled,alignment_status,groundtruth_celltype
0,DLNVKPKTIKTLEENLG,cecedbdbedbeccceb,550348301278027135,A00111:155:HFCWYDMXX:1:1359:1190:25066\tCB:Z:G...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast
1,IKTLEENLGITIQDIGV,edbecccebebeccebe,974148631326387115,A00111:155:HFCWYDMXX:1:1359:1190:25066\tCB:Z:G...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast
3,CRKLKLDPFLTPYTKIN,addedecbfebbfbdec,1662986983738689810,A00111:155:HFCWYDMXX:2:1129:25961:14544\tCB:Z:...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast
4,RKLKLDPFLTPYTKINS,ddedecbfebbfbdecb,799720167227311772,A00111:155:HFCWYDMXX:2:1129:25961:14544\tCB:Z:...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast
7,LDPFLTPYTKINSRWIK,ecbfebbfbdecbdfed,537198855570371996,A00111:155:HFCWYDMXX:2:1360:14886:30185\tCB:Z:...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast


## Remove lemur for now

In [264]:
hash2kmer_no_lemur = hash2kmer.query('species != "lemur"')
hash2kmer_no_lemur.head()

,kmer_in_sequence,kmer_in_alphabet,hashval,read_name,gene_name,species,cell_id,sketch_id,moltype,ksize,scaled,alignment_status,groundtruth_celltype
0,DLNVKPKTIKTLEENLG,cecedbdbedbeccceb,550348301278027135,A00111:155:HFCWYDMXX:1:1359:1190:25066\tCB:Z:G...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast
1,IKTLEENLGITIQDIGV,edbecccebebeccebe,974148631326387115,A00111:155:HFCWYDMXX:1:1359:1190:25066\tCB:Z:G...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast
3,CRKLKLDPFLTPYTKIN,addedecbfebbfbdec,1662986983738689810,A00111:155:HFCWYDMXX:2:1129:25961:14544\tCB:Z:...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast
4,RKLKLDPFLTPYTKINS,ddedecbfebbfbdecb,799720167227311772,A00111:155:HFCWYDMXX:2:1129:25961:14544\tCB:Z:...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast
7,LDPFLTPYTKINSRWIK,ecbfebbfbdecbdfed,537198855570371996,A00111:155:HFCWYDMXX:2:1360:14886:30185\tCB:Z:...,None,human,P3_4__GAGCAGAAGTGGACGT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,Fibroblast


## get diagnostic k-mers for each species and 

In [265]:
hash2kmer_no_lemur.groupby

<bound method DataFrame.groupby of         kmer_in_sequence   kmer_in_alphabet              hashval  \
0      DLNVKPKTIKTLEENLG  cecedbdbedbeccceb   550348301278027135   
1      IKTLEENLGITIQDIGV  edbecccebebeccebe   974148631326387115   
3      CRKLKLDPFLTPYTKIN  addedecbfebbfbdec  1662986983738689810   
4      RKLKLDPFLTPYTKINS  ddedecbfebbfbdecb   799720167227311772   
7      LDPFLTPYTKINSRWIK  ecbfebbfbdecbdfed   537198855570371996   
...                  ...                ...                  ...   
23409  FIWNPRLSKVSHRSSSV  fefcbdebdebddbbbe   989408111628569969   
23410  VSHRSSSVLSHGLCVDT  ebddbbbeebdbeaecb    62515329136865349   
23413  VFLFLSSSLPFLFSLFH  efefebbbebfefbefd  1563197722135732861   
23415  SSSLPFLFSHFPMYSAL  bbbebfefbdfbefbbe   605995314189153464   
23416  PFLFSHFPMYSALIPLL  bfefbdfbefbbeebee   741982259205681112   

                                               read_name gene_name species  \
0      A00111:155:HFCWYDMXX:1:1359:1190:25066\tCB:Z:G...      None   h

In [266]:
for celltype, df in hash2kmer_no_lemur.groupby("broad_group"):
    species_hashes = df.groupby('species').kmer_in_alphabet.apply(lambda x: set(x)).to_dict()
    break

KeyError: 'broad_group'

### Matplotlib venn diagrams

In [ ]:
import matplotlib_venn

In [ ]:
hashes = list(species_hashes.values())
# hashes

In [ ]:
len(hashes)

In [ ]:
celltype

In [ ]:
df.groupby('species').cell_id.nunique()

In [ ]:
matplotlib_venn.venn3(list(species_hashes.values()), set_labels=list(species_hashes.keys()))

### Bat has the most unique hashes per cell type but has the fewest cells??

In [ ]:
hash2kmer_no_lemur.groupby(['species']).apply(lambda x: x.hashval.nunique()/x.cell_id.nunique())

## Try UpSet Plot

In [ ]:
import upsetplot as up

In [ ]:
upsetplot_data = up.from_contents(species_hashes)
upsetplot_data.head()

In [ ]:
upsetplotted = up.plot(upsetplot_data)
upsetplotted['intersections'].set_title(celltype)
upsetplotted

## Do on all celltypes

In [ ]:
for celltype, df in hash2kmer_no_lemur.groupby("broad_group"):
    species_hashes = (
        df.groupby("species").kmer_in_alphabet.apply(lambda x: set(x)).to_dict()
    )
    fig, ax = plt.subplots()

    matplotlib_venn.venn3(
        list(species_hashes.values()), set_labels=list(species_hashes.keys())
    )
    ax.set(title = celltype)
    
    upsetplot_data = up.from_contents(species_hashes)
    upsetplotted = up.plot(upsetplot_data)
    upsetplotted['intersections'].set_title(celltype)
#     upsetplotted

## Do on all celltypes, including lemur

In [ ]:
for celltype, df in hash2kmer_with_predictions.groupby("predicted_compartment"):
    species_hashes = (
        df.groupby("species").kmer_in_alphabet.apply(lambda x: set(x)).to_dict()
    )
    upsetplot_data = up.from_contents(species_hashes)
    upsetplotted = up.plot(upsetplot_data)
    upsetplotted['intersections'].set_title(celltype)
#     upsetplotted

## Do on all celltypes, separate by aligned/unaligned

In [ ]:
hash2kmer_no_lemur['species__alignment_status'] = hash2kmer_no_lemur['species'] + ', ' + hash2kmer_no_lemur['alignment_status']
hash2kmer_no_lemur.head()

In [ ]:
# for celltype, df in hash2kmer_no_lemur.groupby("broad_group"):
#     species_hashes = (
#         df.groupby('species__alignment_status').kmer_in_alphabet.apply(lambda x: set(x)).to_dict()
#     )

#     upsetplot_data = up.from_contents(species_hashes)
#     upsetplotted = up.plot(upsetplot_data, sort_by='cardinality')
#     ax = upsetplotted['intersections']
#     ax.set(title=celltype, yscale='log')
# #     upsetplotted

## Get hashes unaligned in bat

In [111]:
%%time

df_bat_unaligned = df.groupby("hashval").filter(
    lambda x: "bat, unaligned" in set(x["species__alignment_status"])
)
df_bat_unaligned.head()

CPU times: user 4min 40s, sys: 6.86 s, total: 4min 47s
Wall time: 4min 47s


,kmer_in_sequence,kmer_in_alphabet,hashval,read_name,gene_name,species,cell_id,sketch_id,moltype,ksize,scaled,alignment_status,broad_group,species__alignment_status
6,EVLARAIRQEKEIKGIQ,ceebdbedccdcedbec,1355044342132838269,A00111:155:HFCWYDMXX:2:1245:18304:18850\tCB:Z:...,None,human,P3_7__AACCGCGGTTGTACAC,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,T cell,"human, unaligned"
6,WDCKLVQPLWKSVWRFL,fcadeecbefdbefdfe,82630740028545526,A00111:155:HFCWYDMXX:1:1379:18249:8140\tCB:Z:T...,None,human,P3_7__TACTCGCCAGCCTTGG,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,T cell,"human, unaligned"
2,PCSWVGRINIVKMAILP,babfebdeceedebeeb,1049665999009068867,A00111:155:HFCWYDMXX:2:2107:7229:26318\tCB:Z:C...,None,human,P3_7__CTCTACGAGTCACGCC,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,T cell,"human, unaligned"
1,ISNFSKVSGYKINVQKS,ebcfbdebbfdececdb,859122113661270444,A00111:155:HFCWYDMXX:1:2130:13295:10864\tCB:Z:...,None,human,P3_7__CGAATGTAGCGCCTCA,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,T cell,"human, unaligned"
22,WWDCKLVQPLWKSVWRF,ffcadeecbefdbefdf,1029003803404016114,A00111:155:HFCWYDMXX:1:1282:19226:10770\tCB:Z:...,None,human,P3_7__CGATCGGAGATGCCTT,alphabet-dayhoff__ksize-51__scaled-10,dayhoff,51,10,unaligned,T cell,"human, unaligned"


In [92]:
species_hashes.keys()

dict_keys(['bat, aligned', 'bat, unaligned', 'human, aligned', 'human, unaligned', 'mouse, aligned', 'mouse, unaligned'])

In [110]:
hash2kmer_no_lemur.groupby(['species', 'broad_group']).cell_id.nunique()

species  broad_group                    
bat      Alveolar Epithelial Type 2          832
         B cell                              165
         Capillary                          2214
         Dendritic                            98
         Fibroblast                         1382
         Macrophage                         1196
         Monocyte                            969
         Natural Killer T cell               351
         Smooth Muscle and Myofibroblast     120
         T cell                              376
human    Alveolar Epithelial Type 2         3147
         B cell                              174
         Capillary                          2093
         Dendritic                           438
         Fibroblast                          725
         Macrophage                         3344
         Monocyte                            752
         Natural Killer T cell               224
         Smooth Muscle and Myofibroblast     587
         T cell             

### Not *exactly* the same number of cells as originally, but same order of magnitude

Bat:
```
broad_group
Alveolar Epithelial Type 2          832
B cell                              165
Capillary                          2214
Dendritic                            98
Fibroblast                         1382
Macrophage                         1196
Monocyte                            969
Natural Killer T cell               351
Smooth Muscle and Myofibroblast     120
T cell                              376
Name: cell_id, dtype: int64
```

Human: 
```
broad_group
Alveolar Epithelial Type 2         3150
B cell                              174
Capillary                          2093
Dendritic                           438
Fibroblast                          725
Macrophage                         3356
Monocyte                            752
Natural Killer T cell               224
Smooth Muscle and Myofibroblast     587
T cell                             2655
Name: cell_id, dtype: int64
```

# Gene orthology

## Assign figure folder

In [267]:
figure_folder = (
    "/home/olga/googledrive/ImmuneEvolution/figures/kmer_gene_orthology"
)
!mkdir -p $figure_folder

## Use 1:1 orthologs from Anndata of bat-Lemur-human-mouse

In [268]:
describe(adata_shared.var)

(10560, 9)
--- First 5 entries ---


,bat__gene_name,mouse_lemur__gene_name-bat,mouse__gene_name-bat,mouse_lemur__gene_name_x-hlm,mouse__gene_name_x-hlm,gene_ids-lemur-hlm,n_cells-mouse-hlm,mouse_lemur__gene_name_y-hlm,mouse__gene_name_y-hlm
human__gene_name,,,,,,,,,
SAMD11,SAMD11,SAMD11,Samd11,SAMD11,Samd11,gene2117,5381,SAMD11,Samd11
NOC2L,NOC2L,NOC2L,Noc2l,NOC2L,Noc2l,gene2116,48156,NOC2L,Noc2l
KLHL17,KLHL17,KLHL17,Klhl17,KLHL17,Klhl17,gene2115,11262,KLHL17,Klhl17
ISG15,ISG15,ISG15,Isg15,ISG15,Isg15,gene2111,74269,ISG15,Isg15
AGRN,AGRN,AGRN,Agrn,AGRN,Agrn,gene2110,23538,AGRN,Agrn


## Assign k-mer type based on alignment status and gene name

In [269]:
PER_SPECIES_ORTHOLOGOUS_GENES = {
    "bat": set(adata_shared.var["bat__gene_name"]),
    "lemur": set(adata_shared.var["mouse_lemur__gene_name_x-hlm"]),
    "human": set(adata_shared.var.index),
    "mouse": set(adata_shared.var["mouse__gene_name-bat"]),
}
for k, v in PER_SPECIES_ORTHOLOGOUS_GENES.items():
    print(k, len(v))

bat 10560
lemur 10560
human 10560
mouse 10560


In [ ]:
%%time

def is_orthologous(query_gene, species, per_species_orthologous_genes=PER_SPECIES_ORTHOLOGOUS_GENES):
    orthologous_genes = per_species_orthologous_genes[species]
    return query_gene in orthologous_genes


def assign_kmer_category(row):
    if row.alignment_status == 'unaligned':
        return 'Not in reference genome'
    elif pd.isnull(row.gene_name):
        return 'In ref genome, not in a gene'
    elif is_orthologous(row.gene_name, row.species):
        return 'In ref genome, in a 1:1 orthologous gene'
    else:
        return 'In ref genome, not in a 1:1 orthologous gene'

hash2kmer['kmer_category'] = hash2kmer.apply(assign_kmer_category, axis=1)
hash2kmer.groupby(['species', 'kmer_category']).size()

In [ ]:
hash2kmer_with_predictions = hash2kmer.join(celltype_predictions, on='cell_id')
describe(hash2kmer_with_predictions)

In [ ]:
kmer_categories = (
    "Not in reference genome",
    "In ref genome, not in a gene",
    "In ref genome, not in a 1:1 orthologous gene",
    "In ref genome, in a 1:1 orthologous gene",
)

In [ ]:
hash2kmer_with_predictions['kmer_category'] = pd.Categorical(hash2kmer_with_predictions['kmer_category'], categories=kmer_categories, ordered=True)

## Write hash2kmer with predictions and orthology to parquet

In [ ]:
hash2kmer_with_predictions.to_parquet(
    os.path.join(sig_outdir_base, "aggregated-hash2kmer-with-predicted-cells--with-orthology.parquet")
)

In [ ]:
1+1

In [ ]:
%%time

celltype_col = "groundtruth_celltype"

diagnostic_kmers_n_per_category = hash2kmer_with_predictions.groupby(
    ["species", celltype_col, "kmer_category"]
).hashval.nunique()
diagnostic_kmers_n_per_category.name = "percent_kmers"
diagnostic_kmers_n_per_category

In [ ]:
diagnostic_kmers_n_per_category_df = diagnostic_kmers_n_per_category.groupby(level=[0, 1]).apply(lambda x: 100*x/x.sum()).reset_index()
diagnostic_kmers_n_per_category_df

In [ ]:
diagnostic_kmers_n_per_category_df.query(
    '(species == "mouse") and (kmer_category == "Not in reference genome")'
).sort_values('percent_kmers')

In [ ]:
diagnostic_kmers_n_per_category_df.query(
    '(species == "mouse") and (kmer_category == "Not in reference genome")'
).sort_values('percent_kmers')

In [ ]:
diagnostic_kmers_n_per_category_df.query(
    '(species == "lemur") and (kmer_category == "Not in reference genome")'
).sort_values('percent_kmers')

# Plot percentage of kmers in ref genome, etc

In [ ]:
species_order = 'human', 'lemur', 'mouse', 'bat'
diagnostic_kmers_n_per_category_df['species'] = pd.Categorical(diagnostic_kmers_n_per_category_df['species'], categories=species_order, ordered=True)

In [ ]:
g = sns.catplot(
    data=diagnostic_kmers_n_per_category_df,
    col=celltype_col,
    order=species_order,
    col_wrap=5,
    y="species",
    x="percent_kmers",
    hue="kmer_category",
    hue_order=kmer_categories,
    palette='mako',
    kind='bar',
    height=2,
    linewidth=1,
    edgecolor='white'
#     legend=True,
)
g.set_titles('{col_name}')
for ax in g.axes.flat:
    title = ax.get_title()
    if title == 'Smooth Muscle and Myofibroblast':
        ax.set_title('Smooth Muscle\nand Myofibroblast', fontsize=10, pad=-20)
pdf = os.path.join(figure_folder, 'unstacked_barplot__col-celltype__y-species__hue-kmer_category.pdf')
g.savefig(pdf)

In [ ]:
g = sns.catplot(
    data=diagnostic_kmers_n_per_category_df,
    hue=celltype_col,
#     col_wrap=2,
    y="species",
    order=species_order,
    x="percent_kmers",
    col="kmer_category",
    col_order=kmer_categories,
    palette='tab10',
    kind='bar',
    height=2.5,
    sharex=True,
        linewidth=.5,
    edgecolor='white'
#     legend=True,
)
g.set_titles('{col_name}')
for ax in g.axes.flat:
    if ax.is_last_row():
        title = ax.get_title()
        title = title.replace(',', ',\n')
        ax.set_title(title, fontsize=10, pad=-20)
#     if ax.is_first_col():
#         ax.set(xscale='log')
pdf = os.path.join(figure_folder, 'unstacked_barplot__col-kmer_category__y-species__hue-celltype.pdf')
g.savefig(pdf)

In [ ]:
figure_folder

In [ ]:
species_order_no_lemur = ['mouse', 'human', 'bat']

In [ ]:
fig, axes = plt.subplots(ncols=5, nrows=2, figsize=(12, 3), sharex=True, sharey=True)


for (celltype, df), ax in zip(
    diagnostic_kmers_n_per_category_df.groupby(celltype_col), axes.flat
):
    
#     legend = ax.is_last_col() and ax.is_last_row()
    # One liner to create a stacked bar chart.
    sns.histplot(
        df,
        y="species",
        hue="kmer_category",
        weights="percent_kmers",
        multiple="stack",
        palette="mako",
        linewidth=1,
        edgecolor="white",
        legend=False,
        order=species_order_no_lemur,
        ax=ax,
    )
    ax.set(
        ylabel="species",
        xlabel='Percentage',
        title=celltype.replace("and", "\nand"),
        yticks=species_order,
    )
    
# Fix the legend so it's not on top of the bars.
# legend = ax.get_legend()
# legend.set_bbox_to_anchor((1, 1))
sns.despine()
fig.tight_layout()
pdf = os.path.join(figure_folder, 'stacked_barplot__col-celtype__y-species__hue-kmer_category.pdf')
fig.savefig(pdf)

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=5, figsize=(4.5, 6), sharex=True, sharey=True)


for (celltype, df), ax in zip(
    diagnostic_kmers_n_per_category_df.groupby(celltype_col), axes.flat
):
    
#     legend = ax.is_last_col() and ax.is_last_row()
    # One liner to create a stacked bar chart.
    sns.histplot(
        df,
        y="species",
        hue="kmer_category",
        weights="percent_kmers",
        multiple="stack",
        palette="mako",
        linewidth=1,
        edgecolor="white",
        legend=False,
        order=species_order_no_lemur,
        ax=ax,
    )
    ax.set(
        ylabel="species",
        xlabel='Percentage',
        title=celltype.replace("and", "\nand"),
        yticks=species_order,
    )
    
# Fix the legend so it's not on top of the bars.
# legend = ax.get_legend()
# legend.set_bbox_to_anchor((1, 1))
sns.despine()
fig.tight_layout()
pdf = os.path.join(figure_folder, 'stacked_barplot__col-celtype__y-species__hue-kmer_category__two_column.pdf')
fig.savefig(pdf)

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(12, 4), sharex=True, sharey=True)


for (species, df), ax in zip(
    diagnostic_kmers_n_per_category_df.groupby("species"), axes.flat
):
    legend = ax.is_last_col()
    # One liner to create a stacked bar chart.
    sns.histplot(
        df,
        y=celltype_col,
        hue="kmer_category",
        weights="percent_kmers",
        multiple="stack",
        palette="mako",
        linewidth=1,
        edgecolor="white",
        legend=legend,
        ax=ax,
    )
    ax.set(
        ylabel="percentage",
        title=species,
#         yticks=["bat", "human"],
    )
# Fix the legend so it's not on top of the bars.
legend = ax.get_legend()
legend.set_bbox_to_anchor((1, 1))

sns.despine()
fig.tight_layout()

pdf = os.path.join(figure_folder, 'stacked_barplot__col-species__y-celltype__hue-kmer_category.pdf')
fig.savefig(pdf)